In [ ]:
import numpy as np 
import mathplotlib.pyplot as plt 
import pandas as pd 
import pandas_datareader as web 
import datetime as dt 

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Dense, Dropout, LSTM


